In [ ]:
# Mounts drive Google

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Carico la rete

# Exports MobileNet-V3 Small
from tensorflow.keras.models import load_model

MobileNetV3Small_ConvLSTM= load_model('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/ConvLSTM/final_model_fold_5.h5')

# Stampo il summary
MobileNetV3Small_ConvLSTM.summary()


In [ ]:
!pip install tensorflow-model-optimization


In [ ]:
!pip install keras-tuner

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import keras_tuner as kt
from sklearn.model_selection import train_test_split


dataset_dir = '/content/gdrive/MyDrive/Dataset/Data_Augmented'
violence_dir = os.path.join(dataset_dir, 'Violence')
nonviolence_dir = os.path.join(dataset_dir, 'NonViolence')

def extract_frames_from_videos(directory, label, sequence_length=16):
    data = []
    labels = []
    for video_file in os.listdir(directory):
        video_path = os.path.join(directory, video_file)
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (224, 224))
            frames.append(frame)
            if len(frames) == sequence_length:
                data.append(frames)
                labels.append(label)
                frames = []
        cap.release()
    return np.array(data), np.array(labels)


violence_data, violence_labels = extract_frames_from_videos(violence_dir, label=1)
nonviolence_data, nonviolence_labels = extract_frames_from_videos(nonviolence_dir, label=0)


all_data = np.concatenate((violence_data, nonviolence_data), axis=0)
all_labels = np.concatenate((violence_labels, nonviolence_labels), axis=0)

train_data, val_data, train_labels, val_labels = train_test_split(all_data, all_labels, test_size=0.1, random_state=42)


del violence_data, violence_labels, nonviolence_data, nonviolence_labels, all_data, all_labels

def apply_clustering(hp):
    num_clusters = hp.Int('number_of_clusters', 2, 32, step=2)
    centroid_init = hp.Choice('centroid_initialization',
                              values=[tfmot.clustering.keras.CentroidInitialization.RANDOM,
                                      tfmot.clustering.keras.CentroidInitialization.KMEANS_PLUS_PLUS,
                                      tfmot.clustering.keras.CentroidInitialization.LINEAR,
                                      tfmot.clustering.keras.CentroidInitialization.DENSITY_BASED],
                              default=tfmot.clustering.keras.CentroidInitialization.LINEAR)

    print(f'Valutando modello con {num_clusters} cluster e inizializzazione del centroide: {centroid_init}')

    clustering_params = {
        'number_of_clusters': num_clusters,
        'cluster_centroids_init': centroid_init
    }

    def cluster_layer(layer):
        cluster_weights = tfmot.clustering.keras.cluster_weights
        if isinstance(layer, keras.layers.ConvLSTM2D):
            return layer
        else:
            return cluster_weights(layer, **clustering_params)


    clustered_model = keras.models.clone_model(
        MobileNetV3Small_ConvLSTM,
        clone_function=cluster_layer
    )


    clustered_model._clear_callback_list()


    callbacks = [PrintCallback()]


    clustered_model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )


    clustered_model.summary()

    return clustered_model



tuner = kt.GridSearch(
    hypermodel=apply_clustering,
    objective="val_loss",
    overwrite=True,
    max_retries_per_trial=3,
    max_consecutive_failed_trials=8,
)


class PrintCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Fine epoca {epoch}, binary_accuracy: {logs["binary_accuracy"]}, val_binary_accuracy: {logs["val_binary_accuracy"]}')


print("Modello MobileNetV3Small_ConvLSTM definito:")
print(MobileNetV3Small_ConvLSTM.summary())



print(train_data.shape)
print(train_labels.shape)
print(val_data.shape)
print(val_labels.shape)




best_model = tuner.get_best_models(num_models=1)[0]


final_model = tfmot.clustering.keras.strip_clustering(best_model)


converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_model = converter.convert()


clustered_tflite_file = '/content/gdrive/My Drive/Modelli/MobileNet_V3_Small/Clusterizzazione/ConvLSTM/final_clustered_model.tflite'
with open(clustered_tflite_file, 'wb') as f:
    f.write(tflite_model)

print('Ottimizzazione completata e modello salvato come TFLite:', clustered_tflite_file)
